# The Aarseth NBODY family of codes (*drafting from Jan 28 - Feb 28*)

With the 
[passing of Sverre Aarseth on Dec 28, 2024](https://ascl.net/wordpress/2025/01/07/sverre-aarseth-father-of-open-source-stellar-dynamics-has-passed-on-to-a-higher-orbit/),
we remember him by showcasing some examples of his legacy codes in NEMO.

Sverre has always made his code available for anybody to use. His current body of work can still be found at https://people.ast.cam.ac.uk/~sverre/web/pages/nbody.htm  as well as an [entry in ASCL](https://ascl.net/1102.006).
It should also be mentioned that many other derivate works exist. Would be nice to have a listing of those.
NEMO is one such, and here we list the programs available in NEMO:

* [nbody0, nbody00](https://teuben.github.io/nemo/man_html/nbody0.1.html) - version from Binney & Tremaine's "Galactic Dynamics" (1987) book, dubbed the Micky Mouse version by Sverre
* [nbody1](https://teuben.github.io/nemo/man_html/nbody1.1.html), [runbody1](https://teuben.github.io/nemo/man_html/runbody1.1.html) - integrator with variable timestep
* nbody2, runbody2 - Ahmad-Cohen N-body code
* nbody4, runbody4 - hermite N-body code with optional stellar evolution
* nbody5 - Regularized AC N-body code with triple & binary collisions (nbody6 now preferred)
* nbody6, runbody6 - Hermite N-body code with optional stellar evolution
* [firstn](https://teuben.github.io/nemo/man_html/firstn.1.html) -  von Hoerners first N-body code (1960)  
* hermit
* u4tos, stou4 - conversion programs of Sverre's "UNIT4" files to and from NEMO snapshot's
* u3tos - conversion program of Sverre's "UNIT3" file to a NEMO snapshot
* [mkplummer](https://teuben.github.io/nemo/man_html/mkplummer.1.html)  - create a Plummer (1911) N-Body sphere. Algorithm by Aarseth, Henon and Wielen (1974)
and Sverre's 1999 paper ["From NBODY1 to NBODY6: The Growth of an Industry"](https://ui.adsabs.harvard.edu/abs/1999PASP..111.1333A) 
outlines the history behind this series.



# Loading NEMO

We start by loading NEMO in the shell.

As a sanity check we first look to see if $NEMO exist. It should not, unless you shell defines NEMO already.

In [1]:
# check to see if NEMO exists
echo $NEMO

In [2]:
# load NEMO    (your location will likely differ).
source $HOME/NEMO/nemo/nemo_start.sh

In [3]:
# show NEMO and some related things
nemo

NEMO:        /home/teuben/NEMO/nemo  - Version:4.5.2
YAPP:        /xs - default yapp plotting device
git:         Branch:master     Counter:12352      Date: 2025/01/30_17:33:05
python:      /home/teuben/NEMO/nemo/anaconda3/bin/python  - Python 3.12.4
OS_release:  Linux Description:	Pop!_OS 22.04 LTS


### Plotting ?

A minor nuisance of using a bash notebook instead of a python notebook is that you cannot produce the typical interactive matplotlib plots. If we compile NEMO with yapp_pgplot, plots can be saved in **png** format and markdown cells can load them after the cell was executed

# nbody0, nbody00, nbody0_ff, nbody0h4

This code was published in an Appendix of the 1987 (first) edition of Binney & Tremaine's *Galactic Dynamics*. The code
can be found in 
**$NEMO/src/nbody/evolve/aarseth/nbody0**, where several derivatives of this *Micky Mouse* version are available.

In [4]:
man nbody0

man: warning: /home/teuben/NEMO/nemo/man/man1/nbody0.1.pdf: ignoring bogus filename
NBODY0(1NEMO)                                                    NBODY0(1NEMO)

NAME
       nbody0, nbody00, nbody0h4 - direct summation Aarseth N-body integrator

SYNOPSIS
       nbody0  in=snap_in out=snap_out [keyword=value ...]
       nbody00 in=snap_in out=snap_out [keyword=value ...]
       nbody0_ff < input > output
       nbody0h4 < input4 > output4

DESCRIPTION
       nbody0  (and  nbody00)  is  the NEMO adaptation of version 0 (the Micky
       Mouse version) of Aarseth's N-body integrator, as published in Appendix
       4B of Binney & Tremaine (1987).  It is a direct N-body integrator, i.e.
       for each particle it computes the force due to all other N-1 particles,
       hence the computational time grows approximately as N*N.  Although also
       being referred to as a 'toy version', it is a fully  functional  N-body
       integrator.

       Each  particle is followed with its own in

### Creating initial conditions

By default the FORTRAN code is compiled with space for a maximum of 2048 particles. We thus create a Plummer (1911) sphere with 2048 particles. We fix the seed to have reproducable results, and integrate a few crossing times to keep the CPU loaded for a few seconds.

In [5]:
rm -f p2048
mkplummer p2048 2048 seed=123
tsf p2048

char Headline[28] "init_xrandom: seed used 123"
char History[43] "mkplummer p2048 2048 seed=123 VERSION=3.0c"
set SnapShot
  set Parameters
    int Nobj 2048 
    double Time 0.00000 
  tes
  set Particles
    int CoordSystem 66306 
    double Mass[2048] 0.000488281 0.000488281 0.000488281 0.000488281 
      0.000488281 0.000488281 0.000488281 0.000488281 0.000488281 
      0.000488281 0.000488281 0.000488281 0.000488281 0.000488281 
      0.000488281 0.000488281 0.000488281 0.000488281 0.000488281 
      . . .
    double PhaseSpace[2048][2][3] -0.570453 -0.0544111 -0.627691 
      -0.0761437 -0.0984996 0.337867 4.84828 -0.318906 -1.70248 
      0.419009 0.228663 0.171499 0.584347 0.246823 -0.113503 0.0872366 
      0.00176906 -0.340730 0.416242 -0.0460422 -0.188560 -0.739448 
      . . .
  tes
tes


### Comparing FORTRAN and C versions

We first compare the performance of the FORTRAN and C versions. We use **out=.** to not have to write an output file, perhaps saving some overhead.
The default integration time **tcrit=2** is used.

In [6]:
/usr/bin/time nbody0  p2048 . tcrit=2
/usr/bin/time nbody00 p2048 . tcrit=2

### nemo Debug Info: time = 0   steps = 0   energy = -0.244373 cpu =    0.00183 min
### nemo Debug Info: time = 0.25   steps = 19181   energy = -0.244375 cpu =      0.006 min
### nemo Debug Info: time = 0.5   steps = 39859   energy = -0.244383 cpu =       0.01 min
### nemo Debug Info: time = 0.75   steps = 60485   energy = -0.244384 cpu =     0.0138 min
### nemo Debug Info: time = 1   steps = 80889   energy = -0.244386 cpu =     0.0175 min
### nemo Debug Info: time = 1.25   steps = 100950   energy = -0.244386 cpu =     0.0212 min
### nemo Debug Info: time = 1.5   steps = 121284   energy = -0.244389 cpu =     0.0248 min
### nemo Debug Info: time = 1.75   steps = 141176   energy = -0.244396 cpu =     0.0287 min
### nemo Debug Info: time = 2   steps = 161238   energy = -0.244397 cpu =     0.0325 min
1.95user 0.00system 0:01.95elapsed 100%CPU (0avgtext+0avgdata 2688maxresident)k
0inputs+0outputs (0major+330minor)pagefaults 0swaps
### nemo Debug Info: time = 0   steps = 0   energy = -0.2443

### Reproducability

If NEMO's random number generator is working correctly, the number of steps and energy at time=2 should be exactly
```
   time = 2   steps = 161238   energy = -0.244397
```
and although the CPU time varies per machine, my 2023 "Ultra 7 155H" laptop CPU took about 1.7sec for **nbody0** and 2.0sec for **nbody00**. Also notable is that the C version does use a small amount (4%) of system time, whereas FORTRAN took 0. 

### nbody00_ff

This pure FORTRAN version does not have a NEMO CLI. It reads a one line header with 6 numbers from stdin, followed by **n** (the number of bodies) lines containing the mass, position and velocity (7 values per line). The header contains **n,eta,deltat,tcrit,eps2,reset**.   Such an input file can be easily created using basic NEMO tools, which we show below.

In [7]:
# create a fresh Plummer sphere with 5 particles
rm -f p5
mkplummer p5 5 seed=123

# convert the snapshot to the input file that nbody0_ff needs
echo "5 0.02 1.0 10 0.0001 1" > input5
snapprint p5 m,x,y,z,vx,vy,vz format=%.15g >> input5

# run nbody0_ff
nbody0_ff < input5 

# run nbody0, and compare the phase space coordinated at times=10
nbody0 p5 - deltat=1 eps=0.01 tcrit=10 | snaptrim - - times=10 | snapprint -

### nemo Debug Info: m x y z vx vy vz 
 Enter n,eta,deltat,tcrit,eps2,reset:
      0.20        -1.62     -0.19     -0.08        -0.05     -0.31      0.28         0.0881       1
      0.20         3.80     -0.46     -1.16         0.45      0.01      0.11         0.4797       2
      0.20        -0.46      0.11      0.43         0.12     -0.21     -0.40         0.0343       3
      0.20        -0.63     -0.19      0.36        -0.71      0.55     -0.18         0.0347       4
      0.20        -1.08      0.73      0.46         0.19     -0.03      0.20         0.0815       5
     time =   0.00  steps =     0 energy =   -0.1811

      0.20        -1.46     -0.39      0.26         0.42      0.02      0.36         0.0416       1
      0.20         4.23     -0.44     -1.04         0.42      0.02      0.12         0.4820       2
      0.20        -1.16      0.06      0.09        -0.99      0.47     -0.08         0.0361       3
      0.20        -0.82      0.44      0.27        -0.17      0.24   

Did you see something like this?
```
-1.36279 1.16632 -2.41738 -0.19854 0.021288 -0.104094 
7.29869 -0.227201 0.116816 0.283669 0.0275029 0.13166 
-2.38877 -0.457732 0.726767 -0.227988 0.0829789 -0.433205 
-2.22922 -0.465448 0.721034 -0.727559 -0.272988 0.382557 
-1.31639 -0.0168633 0.85151 0.87065 0.141302 0.0230623
```
if so, then it's reproducable.

### Comparing nbody0 and nbody0_ff

Apart from the limited accuracy that nbody0_ff shows, the comparison is excellent, as well as number of steps taken and the energy in the final snapshot:
```
     time = 10   steps = 2351   energy = -0.181269
```

# nbody1, runbody1

This code is an official version from 1997, though minor updates have been tracked up to a 2019 fix to the KZ(15) parameter. Within NEMO the code lives in **$NEMO/src/nbody/evolve/aarseth/nbody1**

This version also introduces the more formal *run* interface in NEMO to allow one to run legacy codes with a NEMO command line interface. It also includes conversion between NEMO's snapshots and NBODYx files (unit3, unit4).

In [8]:
rm -f p128 
mkplummer p128 128

eta=0.1
eps=0.1
for n in 1 2 4 8 16 32 64; do
nbody00 p128 . eta=$eta/$n eps=$eps
done


eta=0.02
eps=0.1
for n in 1 2 4 8 16 32 64; do
nbody00 p128 . eta=$eta eps=$eps tcrit=$n
done


In [9]:
echo nbody00 p128 . eta=$eta eps=$eps tcrit=2

nbody00 p128 . eta= eps= tcrit=2


# nbody2, runbody2


## A Plummer sphere

To create a Plummer (1911) sphere with 128 particles the **mkplummer** program is available.   



In [10]:
rm -f p128 
mkplummer p128 128

The contents is binary.  So viewing contents needs a program **tsf** (*type structured file*)

In [11]:
tsf p128

char Headline[35] "init_xrandom: seed used 1738340776"
char History[32] "mkplummer p128 128 VERSION=3.0c"
set SnapShot
  set Parameters
    int Nobj 128 
    double Time 0.00000 
  tes
  set Particles
    int CoordSystem 66306 
    double Mass[128] 0.00781250 0.00781250 0.00781250 0.00781250 
      0.00781250 0.00781250 0.00781250 0.00781250 0.00781250 
      0.00781250 0.00781250 0.00781250 0.00781250 0.00781250 
      0.00781250 0.00781250 0.00781250 0.00781250 0.00781250 
      . . .
    double PhaseSpace[128][2][3] 0.194839 -0.602592 0.441118 0.667724 
      0.487544 0.0467524 0.370492 0.265740 0.426331 -0.468422 0.536096 
      0.0564449 0.387087 0.683869 -0.851504 0.114660 0.688913 0.110726 
      -0.637686 0.0769993 0.276512 -0.376335 -0.781450 -0.646013 
      . . .
  tes
tes


# nbody4, runbody4

# nbody5

Unlike other versions, this version does not yet have a **runbody5**, essentially because **nbody5** is not maintained, and **nbody6** should be used.  Yet, the manual page talks about a benchmark and it was fun to compare the performance with the numbers from 1995 when this was documented.

In [12]:
man nbody5

NBODY5(1NEMO)                                                    NBODY5(1NEMO)

NAME
       nbody5 - Regularized AC N-body code with triple & binary collisions

SYNOPSIS
       nbody5  <  parameter_file > log_file

DESCRIPTION
       nbody5  is  a  regularized Ahmed-Cohen N-body code with triple & binary
       collisions.

       Maintenance has stopped a while back, nbody6(1NEMO) is the  recommended
       replacement.

PARAMETERS
       The  input  parameters are given in a small ASCII parameter_file (which
       can also be read from standard input) in the order as detailed below

       Example input parameter_file for the  standard  internal  benchmark  (a
       100-body problem integrated for 20 crossing times):
       1 1000.0                                     KSTART TCOMP
       100 5 10 30 35 1                             N NFIX NCRIT NRAND NNBMAX NRUN
       0.02 0.03 0.4 1.0 1.0 20.0 5.0e-05 1.0 0.6   ETAI ETAR RS0 DTADJ DELTAT TCRIT QE RBAR ZMBAR
       1 2 0 0 1 0 1 0

# nbody6, runbody6

In [13]:
man runbody6

RUNBODY6(1NEMO)                                                RUNBODY6(1NEMO)

NAME
       runbody6  -  Hermite  N-body code with optional stellar evolution, NEMO
       frontend to nbody6 and nbody6++

SYNOPSIS
       runbody6 [parameter=value]
       nbody6   < nbody6.in
       nbody6++   < nbody6.in

DESCRIPTION
       runbody6 is a NEMO frontend to the Aarseth family  NBODY6(++)  program.
       It  creates  a  run directory, prepares a parameter input file, and op‐
       tionally input (snapshot(5NEMO) data, and runs the NBODY6(++)  program.
       This can be particularly useful if you need to run a lot of simulations
       unattended.

       Note that this front-end is not yet designed to run every possible  way
       in  which NBODY6++ can be run, particularly re-runs, but it most useful
       to run initial conditions from a NEMO snapshot(5NEMO) file.

       Although the kz= vector can be specified, an alternative way is the in‐
       dexed  parameter  access  method  